# Object Tracking

- Goals
 - Object Tracking techniques
   - Optical Flow
   - MeanShift and CamShift
   - Understand more advanced tracking
   - Review built-in tracking APIs

#### Class 1 Optical Flow

- Optical flow is the pattern of apparent motion of image objects between two consecutive frames caused by the movement of object or camera.
- Assumptions:
 - The pixel intensities of an object do not change between consecutive frames
 - Neighbouring pixels have similar motion.
- The optical flow methods in OpenCV will first take in a given set of points and a frame.
- then it will attempt to find those point sin the next frame.
- It is up to the user to supply the points to track.
- The Lucas-Kanade computes optical flow for a sparse feature set
 - Meaning only the points it was told to track
-  We can use Gunner Farneback's Algorithm to calculate dense optical flow
- This dense optical flow will calculate flow for all points in an image.
- It will color them black if no flow is detected.


In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#corner detection parameters
corner_track_params = dict(maxCorners = 10,
                          qualityLevel = 0.3,
                          minDistance = 7,
                          blockSize = 7)

In [3]:
#parameters for luca-kannade optical flow function
lk_params = dict(winSize = (200,200),
                maxLevel=2,
                criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03))

In [4]:
cap = cv2.VideoCapture(0)

ret,prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY)

#points to track
prevPts = cv2.goodFeaturesToTrack(prev_gray,mask=None,
                                  **corner_track_params)

mask = np.zeros_like(prev_frame)

while True:
    ret,frame = cap.read()
    

    frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    nextPts,status,err = cv2.calcOpticalFlowPyrLK(prev_gray,frame_gray,
                                                prevPts,None,**lk_params)
    
    good_new = nextPts[status == 1]
    good_prev = prevPts[status == 1]
    
    for i,(new,prev) in enumerate(zip(good_new,good_prev)):
        x_new,y_new = new.ravel()
        x_prev,y_prev = prev.ravel()
        
        mask = cv2.line(mask,(x_new,y_new),(x_prev,y_prev),
                        (255,0,0),3)
        
        frame = cv2.circle(frame,(x_new,y_new),6,(0,255,0),-1)
    
    img = cv2.add(frame,mask)
    cv2.imshow('track',img)
    
    k = cv2.waitKey(30) & 0xFF
    
    if k == 27:
        break
        
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()

In [5]:
cap = cv2.VideoCapture(0)

ret,frame1 = cap.read()

prev_img = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255

while True:
    ret,frame2 = cap.read()
    
    nextImg = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prev_img,nextImg,
                                    None,0.5,3,15,3,5,1.2,0)
    
    mag,ang = cv2.cartToPolar(flow[:,:,0],flow[:,:,1],
                              angleInDegrees = True)
    
    hsv_mask[:,:,0] = ang/2
    
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    
    bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    
    cv2.imshow('frame',bgr)
    k = cv2.waitKey(30) & 0xFF
    
    if k == 27:
        break
        
        prev_img = nextImg
        
cv2.destroyAllWindows()
cap.release()
    

#### Class 2 MeanShift and CAMShift

In [6]:
cap = cv2.VideoCapture(0)

ret,frame = cap.read()

face_cascade = cv2.CascadeClassifier('../Data/haarcascades/haarcascade_frontalface_default.xml')

face_rects = face_cascade.detectMultiScale(frame)

(x,y,w,h) = tuple(face_rects[0])

track_window = (x,y,w,h)

roi = frame[y:y+h,x:x+w]

hsv_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)

roi_hist = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])

cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

#termination criteria
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,1)

while True:
    ret,frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        
        #trail 1
        ret,track_window = cv2.meanShift(dst,track_window,term_crit)
        
        x,y,w,h = track_window
        img2 = cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),5)

        #trail 2
#         ret,track_window = cv2.CamShift(dst,track_window,term_crit)
        
#         pts = cv2.boxPoints(ret)
#         pts = np.int0(pts)
#         img2 = cv2.polylines(frame,[pts],True,(255,0,0),4)
        
        cv2.imshow('img',img2)
        
        k = cv2.waitKey(30) & 0xFF
    
        if k == 27:
            break
    else:
        break
        
cv2.destroyAllWindows()
cap.release()

#### Class 3 Tracking API's

In [7]:
def ask_for_tracker():
    print('Welcome! What Tracker API would you like to use?')
    print('Enter 0 for Boosting: ')
    print('Enter 1 for MIL: ')
    print('Enter 2 for KCF: ')
    print('Enter 3 for TLD: ')
    print('Enter 4 for MedianFlow: ')
    choice = input("Please select your tracker: ")
    
    if choice == '0':
        tracker = cv2.TrackerBoosting_create()
    if choice == '1':
        tracker = cv2.TrackerMIL_create()
    if choice == '2':
        tracker = cv2.TrackerKCF_create()
    if choice == '3':
        tracker = cv2.TrackerTLD_create()
    if choice == '4':
        tracker = cv2.TrackerMedianFlow_create()
        
    return tracker

In [ ]:
tracker = ask_for_tracker()
tracker_name = str(tracker).split()[0][1:]

#read video
cap = cv2.VideoCapture(0)

#read first frame
ret,frame = cap.read()

#special function allows us to draw on the very first frame our desired ROI
roi = cv2.selectROI(frame,False)

# Initialize tracker with first frame and bounding box
ret = tracker.init(frame,roi)

while True:
    #read a new frame
    ret,frame =  cap.read()
    
    #update tracker
    success,roi = tracker.update(frame)
    
    #roi variable is a tuple of 4 floats
    #we need each value and we them as integers
    (x,y,w,h) = tuple(map(int,roi))
    
    #Draw rectangle as tracker moves
    if success:
        # tracking success
        p1= (x,y)
        p2 = (x+w,y+h)
        cv2.rectangle(frame,p1,p2,(0,255,0),3)
    else:
        #tracking failure
        cv2.putText(frame,'Failure to Detect tracking!!',
                   (100,200),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),3)
    
    #display tracker type on frame
    cv2.putText(frame,tracker_name,
                   (20,400),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),3)
    
    #display result
    cv2.imshow(tracker_name,frame)
    
    k = cv2.waitKey(1) & 0xFF
    
    if k == 27:
        break
            
cv2.destroyAllWindows()
cap.release()